In [1]:
from datasets import load_dataset
import pandas as pd
import chess
import numpy as np
import tensorflow as tf

subset = load_dataset("laion/strategic_game_chess", data_files="chess_game_0001.parquet")
df = subset["train"].to_pandas()

C:\Users\CODETE\anaconda3\envs\chess\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_moves = df["Moves"].apply(lambda x: [str(i) for i in x])

In [3]:
moves_array = np.array(df_moves)

In [4]:
boards = []
for board_moves in moves_array[:10000]:
    board = chess.Board()
    board_states = []
    for move_str in board_moves[:12]:
        move = chess.Move.from_uci(move_str)
        board.push(move)
        board_states.append(board.copy())
    white_states = board_states[::2]
    boards.append(white_states)

In [5]:
data = np.array(boards).astype(str)

In [23]:
on_hot_mapping = {".": np.array([0,0,0,0,0,0]),
                  "P": np.array([1,0,0,0,0,0]),
                  "N": np.array([0,1,0,0,0,0]),
                  "B": np.array([0,0,1,0,0,0]),
                  "R": np.array([0,0,0,1,0,0]),
                  "Q": np.array([0,0,0,0,1,0]),
                  "K": np.array([0,0,0,0,0,1]),
                  "p": np.array([-1,0,0,0,0,0]),
                  "n": np.array([0,-1,0,0,0,0]),
                  "b": np.array([0,0,-1,0,0,0]),
                  "r": np.array([0,0,0,-1,0,0]),
                  "q": np.array([0,0,0,0,-1,0]),
                  "k": np.array([0,0,0,0,0,-1])}
games_data = []
games_labels = []
for i in range(len(data)):
    for j in range(len(data[i])-1):
        data_split = np.array([row.split() for row in data[i][j].split("\n")])
        labels_split = np.array([row.split() for row in data[i][j+1].split("\n")])
        
        data_encoded_state = np.zeros([8, 8, 6])
        labels_encoded_state = np.zeros([8, 8, 6])
        for char, encoding in on_hot_mapping.items():
                data_encoded_state[data_split == char] = encoding
                labels_encoded_state[labels_split == char] = encoding
        games_data.append(np.array(data_encoded_state))
        games_labels.append(np.array(labels_encoded_state))
train_data = tf.constant(games_data[0 : int(len(games_data) * .70)])
train_labels = tf.constant(games_labels[0 : int(len(games_data) * .70)])
test_data = tf.constant(games_data[int(len(games_data) * .70) : len(games_data)])
test_labels = tf.constant(games_data[int(len(games_data) * .70) : len(games_data)])

In [26]:
train_data.shape, train_labels.shape, test_data.shape, test_labels.shape

(TensorShape([35000, 8, 8, 6]),
 TensorShape([35000, 8, 8, 6]),
 TensorShape([15000, 8, 8, 6]),
 TensorShape([15000, 8, 8, 6]))

In [47]:
'''
baseline_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(8, 8, 6), padding='same', strides=1,),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=1, input_shape=(8, 8, 6)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=1, input_shape=(8, 8, 6)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="softmax", name="output1"),
    tf.keras.layers.Dense(64, activation="softmax", name="output2"),
])
'''

input_layer = tf.keras.layers.Input(shape=(8,8,6))
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=1)(input_layer)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=1)(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=1)(x)
x = tf.keras.layers.Flatten()(x)
output1 = tf.keras.layers.Dense(64, activation="softmax", name="output1")(x)
output2 = tf.keras.layers.Dense(64, activation="softmax", name="output2")(x)

baseline_model = tf.keras.Model(inputs=input_layer, outputs=[output1, output2])

baseline_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
baseline_model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 8, 8, 6)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_36 (Conv2D)  │ (None, 8, 8, 32)  │      1,760 │ input_layer_12[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_37 (Conv2D)  │ (None, 8, 8, 64)  │     18,496 │ conv2d_36[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_38 (Conv2D)  │ (None, 8, 8, 64)  │     36,928 │ conv2d_37[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_12          │ (None, 4096)      │          0 │ conv2d_38[0][0]   │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output1 (Dense)     │ (None, 64)        │    262,208 │ flatten_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output2 (Dense)     │ (None, 64)        │    262,208 │ flatten_12[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 581,600 (2.22 MB)

 Trainable params: 581,600 (2.22 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
bsm_history = baseline_model.fit(train_data, train_labels, epochs=10)

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 8, 8, 6), output.shape=(None, 64)